# Dataset para citas  




---



In [ ]:
!pip install -U tensorflow-text==2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 570.3 MB 347 bytes/s 
     |████████████████████████████████| 14.8 MB 44.6 MB/s 
     |████████████████████████████████| 132 kB 63.3 MB/s 
     |████████████████████████████████| 462 kB 47.4 MB/s 
     |████████████████████████████████| 1.3 MB 44.3 MB/s 
     |████████████████████████████████| 5.6 MB 46.2 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=e9631fe47d51e4f66d4ecb3d7543ac1cb965a472fd6f8f20196e80bac71f997f
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=4030776492d609820447ccaf471fa6f95da20f541a1a656f37c575f3c286317f
  Stored in directory: /root/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c

In [ ]:
!pip install git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-jxzdtvdl
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-jxzdtvdl
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=184397 sha256=b941371fe651a40fdc80276063d5b2e7ce949bd02792641fb206506c851ab8d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-79x8prly/wheels/cc/c4/d8/5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Successfully built tensorflow-docs


In [ ]:
import pandas as pd 
from google.colab import files
import os
from google.colab import drive
from urllib.request import urlopen
import pickle 
from os import remove
import pickle 
import pandas as pd 
from google.colab import files
import tensorflow as tf
import tensorflow_hub as hub 
import tensorflow_text as text
import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers


In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile

In [ ]:
drive.mount('/content/drive') 
os.chdir('/content/drive/MyDrive/APA_entrenamientos')

Mounted at /content/drive


In [ ]:
df = pd.DataFrame({'Categoria':[], 'Cita':[]})

In [ ]:
df.to_csv('citas_dataset2.csv')

In [ ]:
files.download('citas_dataset2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df=pd.read_csv('citas_dataset.csv', encoding='latin-1')

In [ ]:
df.head()

,Categoria,Cita
0,apa,"ANDREU, Alicia Graciela. Juanito Santa Cruz e..."
1,apa,"ARESTI de la Torre, Lore. El miedo y la muert..."
2,apa,"ARGULLOL,Rafael. El héroe y el único. El espír..."
3,apa,"BAJTÍN, Mijaíl M. Problemas de la poética de D..."
4,apa,"BERISTÁIN, Helena. Diccionario de retórica y p..."




---



# Codificando BERT para citas 



---



In [ ]:
df_ieee=df[df['Categoria']=='ieee']
df_apa = df[df['Categoria']=='apa']

In [ ]:
df_apa_submuestra = df_ieee.sample(df_ieee.shape[0])
df_apa_submuestra.shape

(1551, 2)

In [ ]:
df_balanceado = pd.concat([df_ieee, df_apa])
df_balanceado.shape

(2516, 2)

In [ ]:
df_balanceado['Categoria'].value_counts()

ieee    1551
apa      965
Name: Categoria, dtype: int64

In [ ]:
df_balanceado['ieee'] = df_balanceado['Categoria'].apply(lambda x : 1 if x == 'ieee' else 0)

In [ ]:
df_balanceado.sample(5)

,Categoria,Cita,ieee
1248,apa,"-----------------------------------, Jorge Iba...",0
1978,ieee,"39. Gamboa, A. R., F.D. Hernández, E. Guerrero...",1
90,apa,"Díaz-Díaz, J. L., Portela Praza, A. H., Hernán...",0
2384,ieee,"91. J.M. Antunes, L.F. Menezes, J.V. Fernandes...",1
205,ieee,"3. Benjamin C. Kuo (1996), Sistemas de Contro...",1




---



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanceado['Cita'], df_balanceado['ieee'], stratify=df_balanceado['ieee'])

In [ ]:
X_train.head(4)

1337    Narváez, Froylán (1990). Alternativas cultural...
2250    171. Zambrano-M EL, Buitrago FLA, Durán GLA, S...
769     14. Choudhary, T.V., Banerjee., S., y Choudhar...
214     12. Valdés,  Fernando  &  Pallas,  Ramón  (200...
Name: Cita, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
# Bert capas
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# capas red neuronal
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs =[l])
#model = keras.Sequential(
 #   [
  #      layers.Dropout(0.1, name='Dropout'),
   #     layers.Dense(1, activation='sigmoid', name='output')
    #]
#)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'sequence_output':  109482241   keras_layer[2][0]                
                                                                 keras_layer[2][1]                
                                                                 keras_layer[2][2]                
____________________________________________________________________________________________

In [ ]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10)


Epoch 1/10
59/59 [==============================] - 885s 15s/step - loss: 0.5581 - accuracy: 0.7213
Epoch 2/10
59/59 [==============================] - 880s 15s/step - loss: 0.4200 - accuracy: 0.8331
Epoch 3/10
59/59 [==============================] - 879s 15s/step - loss: 0.3492 - accuracy: 0.8718
Epoch 4/10
59/59 [==============================] - 865s 15s/step - loss: 0.3109 - accuracy: 0.8861
Epoch 5/10
59/59 [==============================] - 871s 15s/step - loss: 0.2789 - accuracy: 0.8988
Epoch 6/10
59/59 [==============================] - 878s 15s/step - loss: 0.2611 - accuracy: 0.9083
Epoch 7/10
59/59 [==============================] - 865s 15s/step - loss: 0.2383 - accuracy: 0.9078
Epoch 8/10
59/59 [==============================] - 874s 15s/step - loss: 0.2268 - accuracy: 0.9184
Epoch 9/10
59/59 [==============================] - 866s 15s/step - loss: 0.2162 - accuracy: 0.9210
Epoch 10/10
59/59 [==============================] - 869s 15s/step - loss: 0.2075 - accuracy: 0.9263

In [ ]:
nameRN='Citas_RN.h5'

In [ ]:
model.save(nameRN)

In [ ]:
model.evaluate(X_test,y_test)

20/20 [==============================] - 296s 15s/step - loss: 0.1816 - accuracy: 0.9491


[0.1815643310546875, 0.9491255879402161]

In [ ]:
def interpretacion_cita ( ref ):
  interprete = []
  for cita in ref:
    if cita >=0.5 : 
      interprete.append('cita ieee')
    if cita<0.5 : 
      interprete.append('cita apa')
  return interprete

In [ ]:
citas = [ 
    'González, R. (2013). Costos Paramétricos - México, D.F. Instituto Mexicano de Ingeniería de Costos. D.F, México. Editorial Trillas',
    '1.  SEP, 2011. “Programa de Estudio 2011, Guía para la Educadora, Educación  Básica Prescolar”. México, SEP.',
    '9.  Moltó  ,  E.  Fundamentos  de  la  Educación  en  Física.  Ministerio  de  Educación, La Habana, 2003.',
    'Bloot, S. J., & Pye, K. (2001). “Gradistat: A gran size distribution and statics package for the analysis of unconsolidated sediments”. Earth Surface Processes and Landforms, 261.',
]

In [ ]:
analisis = model.predict(citas)

In [ ]:
interpretacion_cita(analisis)

['cita apa', 'cita ieee', 'cita ieee', 'cita apa']